### ***Import Libraries***

In [13]:
from selenium import webdriver
from selenium.webdriver import ChromeOptions
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from time import sleep
import requests
from lxml import etree
import random
import os

### ***Read Keywords for Image Searching***

In [14]:
keywords_path = 'keywords.txt'
keywords = open(keywords_path, 'r').read().split('\n')

### ***Initialize the Chrome Driver***

Important note for chrome driver setting: https://stackoverflow.com/questions/60362018/macos-catalinav-10-15-3-error-chromedriver-cannot-be-opened-because-the-de

In [16]:
options = ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-automation']) # Use as Developer
service = Service('/usr/local/bin/chromedriver') # Path to chromedriver
browser = webdriver.Chrome(service=service, options=options) # Open Chrome
js = open('stealth.min.js').read() # Load stealth.min.js
browser.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {'source': js}) 

{'identifier': '2'}

### ***Update the URL***

In [17]:
keyword = keywords[1]
browser.get('https://www.google.com.hk/search?q={}+building&tbm=isch'.format(keyword))
sleep(2)

### ***Download the Images***

In [18]:
try:
    # Get the preview images
    img_elements = browser.find_elements(by=By.XPATH, value='//div[@class="wIjY0d jFk0f"]/div')

    # Initialize the count
    count = 0

    scroll_distance = 3    
    
    # Loop through the preview images
    for img_element in img_elements:
        try:
            if count % 10 == 0:
                current_height = browser.execute_script("return document.body.scrollHeight")

                browser.execute_script(f"window.scrollTo(0, {current_height + scroll_distance});")
                # Wait for the page to load
                sleep(1)
            
            # Click the image
            img_element.click()
            # Wait for the image to load
            sleep(3)
            # Get the updated page
            page = browser.page_source
            # Parse the updated page
            tree = etree.HTML(page)
            # Get the image urls from the updated page via xpath
            picUrls = tree.xpath('//a[@role="link"]/img[contains(@src, "http")]/@src')
            
            if picUrls:
                # Get the first image url
                picUrl = picUrls[0]
                # Define the directory path to save the images
                pic_path = './pic/{}'.format(keyword)
                # If the directory does not exist, create it
                if not os.path.exists(pic_path):
                    os.makedirs(pic_path)
                # Save the image to the cooresponding directory
                pic = requests.get(picUrl)
                open(f'{pic_path}/{keyword}_{count}.jpg', 'wb').write(pic.content)
                # Update the count
                count += 1
        except Exception as e:
            print(e)
except Exception as e:
    print(e)

Message: element click intercepted: Element <div jsname="dTDiAc" class="eA0Zlc WghbWd FnEtTd mkpRId m3LIae RLdvSe qyKxnc ivg-i PZPZlf GMCzAd" jsdata="j0Opre;84GahSlvRngHEM;CVG5Ao" jscontroller="Um3BXb" data-attrid="images universal" data-docid="84GahSlvRngHEM" data-lpage="https://www.architecturaldigest.com/gallery/best-gothic-cathedrals" data-ref-docid="q9y9l3oipJ_J9M" style="flex-basis:263px" jsaction="rcuQ6b:npT2md;tuz4af:RrAr1;nptbHe:XEuVS;h5M12e;qWWJ8e:.CLIENT" data-hveid="CGMQAA" data-ved="2ahUKEwiE4NbngNmFAxUvZUEAHesdA4wQM3oECGMQAA">...</div> is not clickable at point (480, 194). Other element would receive the click: <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/ff/Wells_Cathedral_West_Front_Exterior%2C_UK_-_Diliff.jpg/1200px-Wells_Cathedral_West_Front_Exterior%2C_UK_-_Diliff.jpg" jsaction="VQAsE" class="sFlh5c pT0Scc iPVvYb" style="max-width: 1200px; width: 368px; height: 317px; margin: 0px;" alt="Gothic architecture - Wikipedia" jsname="kn3ccd" aria-hidden=

In [20]:
# Get the initial scroll position
last_height = browser.execute_script("return document.body.scrollHeight")

scroll_distance = 50

while True:
    # Get current scroll position
    current_height = browser.execute_script("return document.body.scrollHeight")
    
    # simulate scrolling
    browser.execute_script(f"window.scrollTo(0, {current_height + scroll_distance});")
    # Wait for a while to load the next batch of images
    sleep(2)

    # Get current scroll position
    new_height = browser.execute_script("return document.body.scrollHeight")

    # Check if the page has a "Show more results" button
    page = browser.page_source
    if '顯示更多結果' in page:
        # Try to click the button
        try:
            show_more_button = browser.find_element(by=By.XPATH, value='//input[@value="顯示更多結果"]')
            show_more_button.click()
            # Wait for a while to load the next batch of images
            sleep(random.uniform(1, 3))
        except NoSuchElementException:
            # If not found, just pass
            pass

    # Check if the page has a "You've reached the end of the results" message
    if '你已經看完了所有內容' in page:
        # Find the message
        break
    
    # If the current scroll position is the same as the last one, break the loop
    if new_height == last_height:
        break
    
    # Get the new scroll position
    last_height = new_height

